In [1]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [0]:
path='/content/gdrive/My Drive/'


In [1]:
path="./"

In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D

from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras import regularizers

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.initializers import glorot_normal
from keras.utils import np_utils
from math import ceil,floor
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [3]:
n_bits=40

In [4]:
X_train=np.load(path+'xtrain.npy').reshape(-1,n_bits)
y_train=np.load(path+'ytrain.npy').reshape(-1,2)
X_test=np.load(path+'xtest.npy').reshape(-1,n_bits)
y_test=np.load(path+'ytest.npy').reshape(-1,2)


[0,1] for prime
[1,0] for non prime

## Baseline single layer model

In [5]:
model = Sequential()
model.add(Dense(2,activation='softmax',input_shape=(n_bits,)))
adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = ['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 82        
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________


In [6]:
early_stop=keras.callbacks.callbacks.EarlyStopping(monitor='accuracy', min_delta=0, patience=40, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
reduce_lr=keras.callbacks.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
model.fit(X_train,y_train,epochs=50,callbacks=[reduce_lr,early_stop])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1541984/3400000 [============>.................] - ETA: 1:15 - loss: 0.4703 - accuracy: 0.7650

KeyboardInterrupt: 

In [9]:
model.evaluate(X_test,y_test)[1]

2000/2000 [==============================] - 0s 43us/step


0.7605000138282776

The base line model does surprisingly well with an accuracy of 76%
Lets look at the weights

In [10]:
for layer in model.layers:
    print(layer.get_weights())

[array([[-4.40155044e-02, -1.08361311e-01],
       [-3.82731222e-02, -7.01513290e-02],
       [-1.97470158e-01, -2.27841109e-01],
       [ 2.38424093e-01,  2.24146217e-01],
       [ 1.00622512e-01,  9.18215513e-02],
       [-2.15671852e-01, -2.33546183e-01],
       [-1.57808021e-01, -1.68485984e-01],
       [ 1.55925348e-01,  1.43707648e-01],
       [ 1.36748582e-01,  1.21481977e-01],
       [-2.27177572e-02, -2.13682111e-02],
       [ 1.46830782e-01,  1.42297655e-01],
       [ 8.88569579e-02,  9.18352455e-02],
       [ 5.31717762e-02,  3.91775332e-02],
       [ 9.96120498e-02,  8.89956579e-02],
       [ 8.48817751e-02,  8.28580335e-02],
       [ 1.12035237e-01,  1.12396799e-01],
       [-3.67360213e-03, -2.91331718e-03],
       [ 5.56271002e-02,  5.76730445e-02],
       [-3.85456607e-02, -5.00233509e-02],
       [ 1.52010798e-01,  1.55696616e-01],
       [-8.62822980e-02, -8.69566351e-02],
       [ 3.36920917e-01,  3.30428123e-01],
       [ 4.22859453e-02,  4.28772755e-02],
       [ 1

In [11]:
pred = model.predict_classes(X_test, verbose=0)
print(classification_report([np.where(r==1)[0][0] for r in y_test], pred))

              precision    recall  f1-score   support

           0       1.00      0.52      0.69      1000
           1       0.68      1.00      0.81      1000

    accuracy                           0.76      2000
   macro avg       0.84      0.76      0.75      2000
weighted avg       0.84      0.76      0.75      2000



Precision of predicting a composite number is 1 i.e There are no false positives

## Adding a layer

In [7]:
model = Sequential()
model.add(Dense(32,activation='relu',input_shape=(n_bits,)))
model.add(Dense(2,activation='softmax'))
adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                1312      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 1,378
Trainable params: 1,378
Non-trainable params: 0
_________________________________________________________________


In [8]:
early_stop=keras.callbacks.callbacks.EarlyStopping(monitor='accuracy', min_delta=0, patience=40, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
reduce_lr=keras.callbacks.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
model.fit(X_train,y_train,epochs=200,callbacks=[reduce_lr,early_stop])

Epoch 1/200
500000/500000 [==============================] - 64s 128us/step - loss: 0.4807 - accuracy: 0.7541
Epoch 2/200
500000/500000 [==============================] - 60s 119us/step - loss: 0.4706 - accuracy: 0.7592
Epoch 3/200
500000/500000 [==============================] - 60s 120us/step - loss: 0.4684 - accuracy: 0.7594
Epoch 4/200
500000/500000 [==============================] - 58s 115us/step - loss: 0.4676 - accuracy: 0.7594
Epoch 5/200
500000/500000 [==============================] - 57s 113us/step - loss: 0.4672 - accuracy: 0.7594
Epoch 6/200
500000/500000 [==============================] - 59s 117us/step - loss: 0.4668 - accuracy: 0.7594
Epoch 7/200
500000/500000 [==============================] - 58s 117us/step - loss: 0.4667 - accuracy: 0.7595
Epoch 8/200
500000/500000 [==============================] - 60s 119us/step - loss: 0.4667 - accuracy: 0.7595
Epoch 9/200
500000/500000 [==============================] - 60s 120us/step - loss: 0.4666 - accuracy: 0.7595
Epoch 10/2

KeyboardInterrupt: ignored

In [9]:
model.evaluate(X_test,y_test)[1]

2000/2000 [==============================] - 0s 70us/step


0.7595000267028809

## Adding more layers and increasing traing size to 500000 samples

In [10]:
model = Sequential()
model.add(Dense(256,activation='relu',input_shape=(n_bits,)))
#model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))
adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               10496     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 128)              

In [11]:
early_stop=keras.callbacks.callbacks.EarlyStopping(monitor='accuracy', min_delta=0, patience=40, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
reduce_lr=keras.callbacks.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
model.fit(X_train,y_train,epochs=200,callbacks=[reduce_lr,early_stop])

Epoch 1/200
500000/500000 [==============================] - 118s 235us/step - loss: 0.4686 - accuracy: 0.7585
Epoch 2/200
500000/500000 [==============================] - 117s 234us/step - loss: 0.4747 - accuracy: 0.7543
Epoch 3/200
500000/500000 [==============================] - 118s 235us/step - loss: 0.4689 - accuracy: 0.7584
Epoch 4/200
500000/500000 [==============================] - 117s 234us/step - loss: 0.4851 - accuracy: 0.7407
Epoch 5/200
500000/500000 [==============================] - 117s 233us/step - loss: 0.4967 - accuracy: 0.7276
Epoch 6/200
500000/500000 [==============================] - 117s 233us/step - loss: 0.4697 - accuracy: 0.7585
Epoch 7/200
500000/500000 [==============================] - 116s 232us/step - loss: 0.4669 - accuracy: 0.7594
Epoch 8/200
500000/500000 [==============================] - 117s 233us/step - loss: 0.4669 - accuracy: 0.7594
Epoch 9/200
500000/500000 [==============================] - 116s 232us/step - loss: 0.4669 - accuracy: 0.7594
E

In [12]:
model.evaluate(X_test,y_test)[1]

2000/2000 [==============================] - 0s 85us/step


0.7605000138282776